In [1]:
import pandas as pd
import numpy as np
import math
from qiskit import QuantumCircuit
from qiskit import Aer, execute
import os

### Lectura de archivos de datos
1. Se importa como **'data'** el archivo de datos proporcionado. Los datos contienen los pedidos Train y Test a comparar. Se compararan todos los pedidos Train con los Test.
   - Se descartan las columnas que no se utilizaran en la comparación.

&nbsp;
2. Se importa como **'data denominacion'** el archivo que contiene el nombre real de las piezas codificadas.

In [2]:
data = pd.read_csv('Montabilidad_to_Quantum.csv',delimiter=';', encoding='latin-1')

columnas_a_quitar = ['file_type','metadata_path', 'metadata_date', 'metadata_size', 'query_key', 'text', 'Expediente', 'Código', 'N_Produccion', 'Fecha_Tecnica', 'metadata_file']
data = data.drop(columnas_a_quitar, axis=1)

data_denominacion = pd.read_excel('Codigos Denominacion 447.xlsx')

Se filtran en dos DataFrame diferentes los vectores Test y Train

In [3]:
def filtrado_vectores_train_test(data):
    vectores_train = data[data['Set'] == 'Train']
    vectores_test = data[data['Set'] == 'Test']
    return (vectores_test, vectores_train)

Función para normalizar un vector:  módulo = 1

In [4]:
def normalizar_vector(vector):
    norma = np.linalg.norm(vector)
    return vector/norma

Se le llama padding a la operación de rellenar un vector con 0s hasta llegar a un tamanño deseado.

En este caso necesitamos rellenar los vectores de tal forma que correspondan a al tamaño $2^n$ para poder codificarlos en estados cuánticos. Siendo n el numerpo de qbits utilizados.

In [5]:
def padding(vec, qubits):
    num_componentes_relleno = 2**qubits - len(vec)
    return np.pad(vec, (0, num_componentes_relleno), mode='constant', constant_values = 0)

Función que calcula el producto interno de dos estados cuánticos. 
> **input:** 2 vectores normalizados.

> **output:** producto interno. 0 si los vectores son totalmente diferentes 1 si son iguales.
1. Se comprueba que los vectores tengan el mismo tamaño, se calcula el numero de qbits necesarios para la operación y se aplica el padding a los vectores.
2. Se inicializan los estados cuánticos mediante la función initialize de Qiskit donde  los datos de los vectores se los codifica en amplitudes.
3. Se realizan las mediciones sobre la base canónica mediante la matriz Z de Pauli. Se procesa el resultado para obtener el producto interno.

La ejecución se hace sobre el simulador 'qasm_simulator'


In [6]:
def inner_prod(vec1, vec2):

    if len(vec1) != len(vec2):
        raise ValueError('Lengths of states are not equal')

    numero_componentes = len(vec1)
    nqubits = math.ceil(np.log2(numero_componentes))

    vec1 = padding(vec1, nqubits)
    vec2 = padding(vec2, nqubits)

    circ = QuantumCircuit(nqubits+1,1)
    vec = np.concatenate([vec1,vec2])/np.sqrt(2)

    circ.initialize(vec, range(nqubits+1))
    circ.h(nqubits)
    circ.measure(nqubits,0)

    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, shots=20000)

    result = job.result()
    outputstate = result.get_counts(circ)

    if ('0' in outputstate.keys()):
        m_sum = float(outputstate["0"])/20000
    else:
        m_sum = 0

    inner_product = 2*m_sum-1
    return inner_product

In [7]:
def correlacion_vectores(vector_1, vector_2):
    test_normalizado = normalizar_vector(vector_1.iloc[ :-4].to_numpy())
    train_normalizado = normalizar_vector(vector_2.iloc[ :-4].to_numpy())
    return inner_prod(test_normalizado, train_normalizado)

In [8]:
def itera_por_matriz_cuantica(vectores_test, vectores_train):
  for index_test, test in vectores_test.iterrows():
    for index_train, train in vectores_train.iterrows():
      correlacion_entre_variables = correlacion_vectores(test, train)
      yield [f'test_{index_test}', f'train_{index_train}', correlacion_entre_variables, test['Categoria'], test['N_Pedido'],train['N_Pedido']]

Funciones para obtener la difernecia de componentes entre el vector Test exótico y el Train similar

In [9]:
def diferencias_componentes(exotico, similar):
    exotico = exotico.apply(lambda x: 1 if x != 0 else 0)
    similar = similar.apply(lambda x: 1 if x != 0 else 0)

    # Obtener las columnas que son 1 en `exotico` y 0 en `similar`
    condicion1 = (exotico == 1) & (similar == 0)
    diferencias_exotico = exotico[condicion1].index.tolist()

    # Obtener las columnas que son 1 en `similar` y 0 en `exotico`
    condicion2 = (similar == 1) & (exotico == 0)
    diferencias_similar = similar[condicion2].index.tolist()
    
    return diferencias_exotico,diferencias_similar

In [10]:
def obtener_denominacion(comp, data_denominacion):
    # Verifica si hay coincidencias
    if comp in data_denominacion['Codigo'].values:
        denominacion = data_denominacion[data_denominacion['Codigo'] == comp]['Denominación'].values[0]
    else:
        denominacion = 'No ha habido coincidencias'
    return denominacion

Función de creación del la tabla final

In [11]:
def creacion_resultado_final( exotico, similar, diferencia_exotico, diferencia_similar):
    # Crear el DataFrame
    columnas = [
        'Exotico',
        'Tipo_Vehículo',
        'Similar',
        'Descripción',
        'Componentes',
        'Id',
        'Categoria_exótico',
        'Categoria_similar',
        'Denominación'
    ]

    data = []

    # Iterar sobre la lista de diferencia exotico
    for comp in diferencia_exotico:
        entrada = [
            exotico['N_Pedido'],
            exotico['Tipo_vehículo'],
            similar['N_Pedido'],
            'Componentes en el exotico que no aparecen en el similar',
            'Mas',
            comp,
            exotico['Categoria'],
            similar['Categoria'],
            obtener_denominacion(comp, data_denominacion)
        ]
        data.append(entrada)

    # Iterar sobre la lista de diferencia similar
    for comp in diferencia_similar:
        entrada = [
            exotico['N_Pedido'],
            exotico['Tipo_vehículo'],
            similar['N_Pedido'],
            'Componentes en el similar que no aparecen en el exotico',
            'Menos',
            comp,
            exotico['Categoria'],
            similar['Categoria'],
            obtener_denominacion(comp, data_denominacion)
        ]
        data.append(entrada)

    # Crear el DataFrame final
    return pd.DataFrame(data, columns=columnas)

In [12]:
def resultados_csv(ruta_guardado, resultado, nombre_resultado):
    # Comprobar si la ruta de guardado existe, si no, crearla
    if not os.path.exists(ruta_guardado):
        os.makedirs(ruta_guardado)

    # Unir la ruta de guardado con el nombre del archivo y la extensión .csv
    ruta_completa = os.path.join(ruta_guardado, f"{nombre_resultado}.csv")

    # Guardar el DataFrame en el archivo CSV
    resultado.to_csv(ruta_completa, index=False)


In [13]:
ruta_guardado = 'D:\Project\I3B\Resultados'

columnas = ['Nombre_Test', 'Nombre_Train', 'Correlacion', 'Categoria', 'N_Pedido_test', 'N_Pedido_train']
categorias = data.groupby('Categoria')

for nom_categoria, categoria in categorias:
    if nom_categoria != 'CLASE V':
         vectores_test, vectores_train = filtrado_vectores_train_test(categoria) 
         
         resultados_correlacion = pd.DataFrame(itera_por_matriz_cuantica(vectores_test, vectores_train), columns=columnas)
         resultados_correlacion_ordenado = resultados_correlacion.sort_values(by='Correlacion')

         exotico = pd.Series(vectores_test[vectores_test['N_Pedido'] == resultados_correlacion_ordenado.iloc[0]['N_Pedido_test']].iloc[0])
         
         similar = resultados_correlacion[resultados_correlacion['N_Pedido_test'] == exotico['N_Pedido']].sort_values(by='Correlacion').iloc[-1]
         train_similar = pd.Series(vectores_train[similar['N_Pedido_train'] == vectores_train['N_Pedido']].iloc[0])
         
         diferencias_exotico, diferencias_similar = diferencias_componentes(exotico.iloc[ :-4], train_similar.iloc[ :-4])

         resultado = creacion_resultado_final(exotico, train_similar, diferencias_exotico, diferencias_similar)

         nombre_resultado = f'resultados_{nom_categoria}'
         resultados_csv(ruta_guardado,resultado,nombre_resultado)
         
         display(resultado)

,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,805,AVANTGARDE,AVANTGARDE,No ha habido coincidencias
1,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,CK8,AVANTGARDE,AVANTGARDE,LENKRADHEIZUNG
2,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,E7M,AVANTGARDE,AVANTGARDE,MBUX MULTIMEDIASYSTEM
3,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,ES2,AVANTGARDE,AVANTGARDE,CAJA DE ENCHUFE DE 12V EN PORTAEQ./ ESPAC.DE C...
4,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,F2W,AVANTGARDE,AVANTGARDE,PAQUETE DE INVIERNO
5,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,HI4,AVANTGARDE,AVANTGARDE,ZONA CLIMATICA 4 (TROPICO)
6,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,IH5,AVANTGARDE,AVANTGARDE,HEADUNIT ROW-AFRIKA/SO-ASIEN/INDIEN/AUS/GOLFST.
7,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,IL2,AVANTGARDE,AVANTGARDE,CODIGO DE EXPORTACION
8,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,J59,AVANTGARDE,AVANTGARDE,DISPOSITIVO DE AVISO PARA LIMITACION DE VELOCIDAD
9,5329280040,447-8-1,5129280341,Componentes en el exotico que no aparecen en e...,Mas,JX1,AVANTGARDE,AVANTGARDE,INTERVALO DE MANTENIMIENTO 25.000 KM


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,D14,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,BARRAS LONGITUDINALES EN EL TECHO ANODIZADAS
1,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,E2Q,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,MBUX AUGMENTED REALITY FUER NAVIGATION
2,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,E7H,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,MBUX MULTIMEDIASYSTEM HIGH
3,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,ES3,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,TOMAS DE CORRIENTE 12 V PARA FILAS TRASERAS DE...
4,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,ES9,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,KIT DE CALES PARA MEDIA INTERFACE
5,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,F71,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,ESPELHO INTERIOR AUTOMATICAMENTE DIMMING
6,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,HI4,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,ZONA CLIMATICA 4 (TROPICO)
7,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,IL6,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,PINTURA METALIZADA
8,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,LX6,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,AFRICA
9,5329233118,447-8-2,5-30032158,Componentes en el exotico que no aparecen en e...,Mas,R18,AVANTGARDE LINE ELECTRIC,AVANTGARDE LINE ELECTRIC,"LEICHTMETALLRAEDER 7,5 J X 18, 10-SPEICHEN DESIGN"


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5-30033331,447-6-1,5229235300,Componentes en el exotico que no aparecen en e...,Mas,X30,BASE,BASE,"CERTIFICADO DE MATRICULACION, PARTE II"
1,5-30033331,447-6-1,5229235300,Componentes en el exotico que no aparecen en e...,Mas,XC9,BASE,BASE,DOCUMENTO COC
2,5-30033331,447-6-1,5229235300,Componentes en el similar que no aparecen en e...,Menos,IL7,BASE,BASE,"PINTURA ACABADO, 1 CAPA ADICIONAL"
3,5-30033331,447-6-1,5229235300,Componentes en el similar que no aparecen en e...,Menos,XW2,BASE,BASE,SUPRESION CERTIFICADO DE CONFORMIDAD
4,5-30033331,447-6-1,5229235300,Componentes en el similar que no aparecen en e...,Menos,XW3,BASE,BASE,"SUPRESION CERTIFICADO DE MATRICULACION, PARTE II"


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,CF8,EXCLUSIVE,EXCLUSIVE,TREN DE RODAJE DEPORTIVO
1,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,D12,EXCLUSIVE,EXCLUSIVE,PORTAEQUIPAJES
2,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,EI0,EXCLUSIVE,EXCLUSIVE,SISTEMA INALAMBRICO DE CARGA DISPOSITIVOS MOVILES
3,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,ES9,EXCLUSIVE,EXCLUSIVE,KIT DE CALES PARA MEDIA INTERFACE
4,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,F3T,EXCLUSIVE,EXCLUSIVE,ZIERELEMENTE HOLZOPTIK OLIVE DUNKELGRAU OFFENP...
5,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,FG0,EXCLUSIVE,EXCLUSIVE,CONSOLA CENTRAL CON TAPA DE TIPO PERSIANA
6,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,FY1,EXCLUSIVE,EXCLUSIVE,PAQUETE DE PROTECCION ANTIRROBO
7,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,FZ9,EXCLUSIVE,EXCLUSIVE,DOS LLAVES PRINCIPALES ADICIONALES
8,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,IN2,EXCLUSIVE,EXCLUSIVE,DISTANCIA ENTRE EJES 3200 MM VOLADIZO LARGO
9,5-30032059,447-8-1,5-30032152,Componentes en el exotico que no aparecen en e...,Mas,J1W,EXCLUSIVE,EXCLUSIVE,INTERVALO DE MANTENIMIENTO 1 AÑO


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,553,MARCO POLO,MARCO POLO,No ha habido coincidencias
1,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,EI0,MARCO POLO,MARCO POLO,SISTEMA INALAMBRICO DE CARGA DISPOSITIVOS MOVILES
2,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,ES9,MARCO POLO,MARCO POLO,KIT DE CALES PARA MEDIA INTERFACE
3,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,F3T,MARCO POLO,MARCO POLO,ZIERELEMENTE HOLZOPTIK OLIVE DUNKELGRAU OFFENP...
4,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,FZ5,MARCO POLO,MARCO POLO,ALARMA ANTIRROBO
5,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,HY1,MARCO POLO,MARCO POLO,RADIOTELEMANDO CALEFACCION ADICIONAL AGUA CALI...
6,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,LB9,MARCO POLO,MARCO POLO,LUCES DE SALIDA
7,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,M72,MARCO POLO,MARCO POLO,FAHRZEUG HVO FAEHIG
8,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,U73,MARCO POLO,MARCO POLO,APOYABRAZOS EN ZONA PASAJEROS
9,5-30033422,447-8-1,5229235363,Componentes en el exotico que no aparecen en e...,Mas,US7,MARCO POLO,MARCO POLO,ASIENTO INDIVIDUAL PRIMERA FILA DERECHA


...

,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5329233109,447-6-2,5353760499,Componentes en el exotico que no aparecen en e...,Mas,E7B,SELECT,SELECT,PREEQUIPO NAVEGACION
1,5329233109,447-6-2,5353760499,Componentes en el exotico que no aparecen en e...,Mas,HI4,SELECT,SELECT,ZONA CLIMATICA 4 (TROPICO)
2,5329233109,447-6-2,5353760499,Componentes en el exotico que no aparecen en e...,Mas,IH5,SELECT,SELECT,HEADUNIT ROW-AFRIKA/SO-ASIEN/INDIEN/AUS/GOLFST.
3,5329233109,447-6-2,5353760499,Componentes en el exotico que no aparecen en e...,Mas,IN2,SELECT,SELECT,DISTANCIA ENTRE EJES 3200 MM VOLADIZO LARGO
4,5329233109,447-6-2,5353760499,Componentes en el exotico que no aparecen en e...,Mas,J1W,SELECT,SELECT,INTERVALO DE MANTENIMIENTO 1 AÑO
...,...,...,...,...,...,...,...,...,...
81,5329233109,447-6-2,5353760499,Componentes en el similar que no aparecen en e...,Menos,XQ1,SELECT,SELECT,VIN DATOS VEHICULO CODIFICADOS CON CIFRA VERIFIC.
82,5329233109,447-6-2,5353760499,Componentes en el similar que no aparecen en e...,Menos,XW9,SELECT,SELECT,CODIGO DE CONTROL MODIFICACION WLTP
83,5329233109,447-6-2,5353760499,Componentes en el similar que no aparecen en e...,Menos,YF7,SELECT,SELECT,EXTINTOR 1KG
84,5329233109,447-6-2,5353760499,Componentes en el similar que no aparecen en e...,Menos,Z2U,SELECT,SELECT,GUARD 360 GRAD


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5-30032073,447-8-1,5129280336,Componentes en el exotico que no aparecen en e...,Mas,805,STYLE,STYLE,No ha habido coincidencias
1,5-30032073,447-8-1,5129280336,Componentes en el exotico que no aparecen en e...,Mas,BS1,STYLE,STYLE,PINZAS DE FRENO CON DISTINTIVO MERCEDES-BENZ
2,5-30032073,447-8-1,5129280336,Componentes en el exotico que no aparecen en e...,Mas,C1L,STYLE,STYLE,PARACHOQUES FRONTAL AMG CON ELEM. DE ADORNO NE...
3,5-30032073,447-8-1,5129280336,Componentes en el exotico que no aparecen en e...,Mas,CA3,STYLE,STYLE,AKTIVER LENK-ASSISTENT
4,5-30032073,447-8-1,5129280336,Componentes en el exotico que no aparecen en e...,Mas,CF8,STYLE,STYLE,TREN DE RODAJE DEPORTIVO
...,...,...,...,...,...,...,...,...,...
73,5-30032073,447-8-1,5129280336,Componentes en el similar que no aparecen en e...,Menos,VD5,STYLE,STYLE,TECHO INTERIOR GRIS CRISTAL
74,5-30032073,447-8-1,5129280336,Componentes en el similar que no aparecen en e...,Menos,VL2,STYLE,STYLE,ALFOMBRILLAS DE TERCIOPELO
75,5-30032073,447-8-1,5129280336,Componentes en el similar que no aparecen en e...,Menos,X31,STYLE,STYLE,PREPARACION DE LA FICHA TECNICA (ALEMANA)
76,5-30032073,447-8-1,5129280336,Componentes en el similar que no aparecen en e...,Menos,XW2,STYLE,STYLE,SUPRESION CERTIFICADO DE CONFORMIDAD


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5329233116,447-7-2,5229235319,Componentes en el exotico que no aparecen en e...,Mas,E2L,TOURER PRO,TOURER PRO,AC LADEN 11 KW / DC LADEN 50 KW
1,5329233116,447-7-2,5229235319,Componentes en el exotico que no aparecen en e...,Mas,ES3,TOURER PRO,TOURER PRO,TOMAS DE CORRIENTE 12 V PARA FILAS TRASERAS DE...
2,5329233116,447-7-2,5229235319,Componentes en el exotico que no aparecen en e...,Mas,F71,TOURER PRO,TOURER PRO,ESPELHO INTERIOR AUTOMATICAMENTE DIMMING
3,5329233116,447-7-2,5229235319,Componentes en el exotico que no aparecen en e...,Mas,FY1,TOURER PRO,TOURER PRO,PAQUETE DE PROTECCION ANTIRROBO
4,5329233116,447-7-2,5229235319,Componentes en el exotico que no aparecen en e...,Mas,FY7,TOURER PRO,TOURER PRO,RADIOTELEMANDO MULTITECLA
...,...,...,...,...,...,...,...,...,...
62,5329233116,447-7-2,5229235319,Componentes en el similar que no aparecen en e...,Menos,XQ1,TOURER PRO,TOURER PRO,VIN DATOS VEHICULO CODIFICADOS CON CIFRA VERIFIC.
63,5329233116,447-7-2,5229235319,Componentes en el similar que no aparecen en e...,Menos,XU1,TOURER PRO,TOURER PRO,ROTULOS / FOLLETOS EN ALEMAN
64,5329233116,447-7-2,5229235319,Componentes en el similar que no aparecen en e...,Menos,XW2,TOURER PRO,TOURER PRO,SUPRESION CERTIFICADO DE CONFORMIDAD
65,5329233116,447-7-2,5229235319,Componentes en el similar que no aparecen en e...,Menos,Z3R,TOURER PRO,TOURER PRO,PREMIUMSCHUTZ REIFEN (3 JAHRE)


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5229233110,447-7-2,5329280024,Componentes en el exotico que no aparecen en e...,Mas,E7B,TOURER SELECT,TOURER SELECT,PREEQUIPO NAVEGACION
1,5229233110,447-7-2,5329280024,Componentes en el exotico que no aparecen en e...,Mas,F71,TOURER SELECT,TOURER SELECT,ESPELHO INTERIOR AUTOMATICAMENTE DIMMING
2,5229233110,447-7-2,5329280024,Componentes en el exotico que no aparecen en e...,Mas,FF1,TOURER SELECT,TOURER SELECT,CONSOLA CENTRAL CON COMPARTIMENTO PORTAOBJETOS
3,5229233110,447-7-2,5329280024,Componentes en el exotico que no aparecen en e...,Mas,HH9,TOURER SELECT,TOURER SELECT,AIRE ACONDICIONADO REGUL. SEMIAUTOMATICA TEMPM...
4,5229233110,447-7-2,5329280024,Componentes en el exotico que no aparecen en e...,Mas,HI4,TOURER SELECT,TOURER SELECT,ZONA CLIMATICA 4 (TROPICO)
...,...,...,...,...,...,...,...,...,...
83,5229233110,447-7-2,5329280024,Componentes en el similar que no aparecen en e...,Menos,XW5,TOURER SELECT,TOURER SELECT,SUPRESION ROTULOS TRASEROS
84,5229233110,447-7-2,5329280024,Componentes en el similar que no aparecen en e...,Menos,XW9,TOURER SELECT,TOURER SELECT,CODIGO DE CONTROL MODIFICACION WLTP
85,5229233110,447-7-2,5329280024,Componentes en el similar que no aparecen en e...,Menos,YF7,TOURER SELECT,TOURER SELECT,EXTINTOR 1KG
86,5229233110,447-7-2,5329280024,Componentes en el similar que no aparecen en e...,Menos,Z3R,TOURER SELECT,TOURER SELECT,PREMIUMSCHUTZ REIFEN (3 JAHRE)


,Exotico,Tipo_Vehículo,Similar,Descripción,Componentes,Id,Categoria_exótico,Categoria_similar,Denominación
0,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,CF7,TOURIER,TOURIER,TREN DE RODAJE DE CONFORT
1,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,ES9,TOURIER,TOURIER,KIT DE CALES PARA MEDIA INTERFACE
2,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,FF1,TOURIER,TOURIER,CONSOLA CENTRAL CON COMPARTIMENTO PORTAOBJETOS
3,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,FY1,TOURIER,TOURIER,PAQUETE DE PROTECCION ANTIRROBO
4,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,FY7,TOURIER,TOURIER,RADIOTELEMANDO MULTITECLA
5,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,IL6,TOURIER,TOURIER,PINTURA METALIZADA
6,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,IN2,TOURIER,TOURIER,DISTANCIA ENTRE EJES 3200 MM VOLADIZO LARGO
7,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,J1W,TOURIER,TOURIER,INTERVALO DE MANTENIMIENTO 1 AÑO
8,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,JX0,TOURIER,TOURIER,SERVICIO POR TRAYECTOS CORTOS
9,5229235295,447-7-1,5-30032155,Componentes en el exotico que no aparecen en e...,Mas,JX1,TOURIER,TOURIER,INTERVALO DE MANTENIMIENTO 25.000 KM
